# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

**ANSWER:**
 **Overview of Understanding the Data from Materials and Methods section:** 
 - Data is from a **Portuguese bank’s campaigns**, conducted between May 2008 and November 2010. 
 - **Primarily, through phone**  and online banking (occasionally).
 - Total of **17 campaigns**, that has 79,354 customer contacts, each with multiple attributes indicating success.
 - Overall, **6,499 contacts resulted in successful subscriptions** to a long-term deposit product, i.e. an **8% success rate**.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from plotly.subplots import make_subplots
from sklearn.model_selection import StratifiedKFold, cross_val_predict, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.feature_selection import SelectFromModel

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

### Business Objective: Predict which client(s) are more likely to subscribe to a term deposit (by utilizing the dataset).
- Based on features such as age, job, marital status, loan(s), education, previous contact including mode and time of contact information. This can help the bank target potential customers more effectively and improve marketing strategies.

In [5]:
print("\nCheck for missing values \n", df.isnull().sum())
df.describe()


Check for missing values 
 age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64


,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [7]:
vc = df['age'].value_counts().reset_index()
vc.columns = ['Age', 'Count']
fig = px.bar(vc, x='Age', y='Count', color='Age', text='Count')
fig.update_layout(
    title_x=0.5,
)

/opt/anaconda3/lib/python3.13/site-packages/kaleido/_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




In [8]:
vc2 = df['job'].value_counts().reset_index()
vc2.columns = ['Job', 'Count']
fig = px.pie(vc2, names='Job', values='Count', hole=0.5, color_discrete_sequence=px.colors.sequential.RdBu)
fig.update_layout(title_x=0.5, showlegend=True, legend_title_text='<b>Job</b>', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))

In [9]:
vc2 = df['marital'].value_counts().reset_index()
vc2.columns = ['Marital-Status', 'Count']
fig = px.pie(vc2, names='Marital-Status', values='Count', hole=0.5, color_discrete_sequence=['Red','Green','Yellow','Blue'])
fig.update_layout(title_x=0.5, showlegend=True, legend_title_text='<b>Marital-Status</b>', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))


In [10]:
vc2 = df['education'].value_counts().reset_index()
vc2.columns = ['Education', 'Count']
fig = px.pie(vc2, names='Education', values='Count', hole=0.5, color_discrete_sequence=px.colors.sequential.RdBu)
fig.update_layout(title_x=0.5, showlegend=True, legend_title_text='<b>Education</b>', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))

In [11]:
vc_def = df['default'].value_counts().reset_index()
vc_def.columns = ['Default', 'Count']
vc_hous = df['housing'].value_counts().reset_index()
vc_hous.columns = ['Housing', 'Count']
vc_loan = df['loan'].value_counts().reset_index()
vc_loan.columns = ['Loan', 'Count']

fig = make_subplots(rows=1, cols=3, specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}]], subplot_titles=('Defaulter?', 'Has Housing Loan', 'Has Personal Loan'))
fig.add_trace(go.Pie(values=vc_def['Count'], labels=vc_def['Default'], hole=0.7, showlegend=False), row=1, col=1)
fig.add_trace(go.Pie(values=vc_hous['Count'], labels=vc_hous['Housing'], hole=0.7, showlegend=False), row=1, col=2)
fig.add_trace(go.Pie(values=vc_loan['Count'], labels=vc_loan['Loan'], hole=0.7, showlegend=True), row=1, col=3)
fig.update_layout(title_x=0.5, showlegend=True, title_text='<b> Credit and Loan Analysis</b>')

In [12]:
vc2 = df['contact'].value_counts().reset_index()
vc2.columns = ['Contact', 'Count']
fig = px.pie(vc2, names='Contact', values='Count', hole=0.5, color_discrete_sequence=px.colors.sequential.RdBu)
fig.update_layout(title_x=0.5, showlegend=True, legend_title_text='<b>Contact</b>', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))

In [13]:
vc = df['campaign'].value_counts().reset_index()
vc.columns = ['Campaign', 'Count']
fig = px.bar(vc, x='Campaign', y='Count', color='Campaign', text='Count')
fig.update_layout(
    title_x=0.5,
)

In [14]:
# create a copy of dataframe called ndf which will only have numerical columns
ndf = df.copy(deep=True)

# Mapping dictionaries for categorical variables
njob = {"unknown": 400, "admin.": 1, "blue-collar": 2, "technician": 3, "services": 4, "management": 5, "retired": 6, "entrepreneur": 7,
    "self-employed": 8,    "housemaid": 9,    "unemployed": 10,    "student": 11}
nmaritial = {    "married": 1,    "single": 2,    "divorced": 3,    "unknown": 4}
neducation = {"university.degree": 1, "high.school": 2, "basic.9y": 3, "professional.course": 4, "basic.4y": 5, "basic.6y": 6, "unknown": 0, "illiterate": 0}
ndefault = {"unknown": 400, "no": 0, "yes": 1}
nhousing = {"unknown": 400, "no": 0, "yes": 1}
nloan = {"unknown": 400, "no": 0, "yes": 1}
ncontact = {"telephone": 1, "cellular": 2}
npoutcome = {"failure": 0, "success": 1, "nonexistent": 2}
ny = {"no": 0, "yes": 1}

# Function to encode categorical columns into numerical using mapping dictionaries
def encode_job(series, column_number):
    return series.map(column_number).fillna(0).astype(int)

# convert categorical columns to numerical:
ndf["njob"] = encode_job(ndf["job"], njob)
ndf["nmarital"] = encode_job(ndf["marital"], nmaritial)
ndf["neducation"] = encode_job(ndf["education"], neducation)
ndf["ndefault"] = encode_job(ndf["default"], ndefault)
ndf["nhousing"] = encode_job(ndf["housing"], nhousing)
ndf["nloan"] = encode_job(ndf["loan"], nloan)
ndf["ncontact"] = encode_job(ndf["contact"], ncontact)
ndf["npoutcome"] = encode_job(ndf["poutcome"], npoutcome)
ndf["nsubscribed"] = encode_job(ndf["y"], ny)

ndf = ndf.drop(columns=["emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed","job","marital","education","default","housing","loan","contact","poutcome","y"])
ndf.head()

,age,month,day_of_week,duration,campaign,pdays,previous,njob,nmarital,neducation,ndefault,nhousing,nloan,ncontact,npoutcome,nsubscribed
0,56,may,mon,261,1,999,0,9,1,5,0,0,0,1,2,0
1,57,may,mon,149,1,999,0,4,1,2,400,0,0,1,2,0
2,37,may,mon,226,1,999,0,4,1,2,0,1,0,1,2,0
3,40,may,mon,151,1,999,0,1,1,6,0,0,0,1,2,0
4,56,may,mon,307,1,999,0,4,1,2,0,0,1,1,2,0


### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [15]:
features = ["age", "njob", "nmarital", "neducation", "campaign", "pdays", "previous", "ndefault", "nhousing", "nloan", "ncontact", "npoutcome"]
X = ndf[features].copy()
y = ndf["nsubscribed"].copy()

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y, test_size=0.4)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [17]:
baseline_accuracy = y.value_counts(normalize=True).max()
print("Baseline accuracy:", baseline_accuracy)

Baseline accuracy: 0.8873458288821987


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [18]:
summary = []

# Start with Simple Model LOGISTIC REGRESSION
logReg = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression())
    ])
start = time.perf_counter()
logReg.fit(X_train, y_train)
train_time = time.perf_counter() - start
train_acc = accuracy_score(y_train, logReg.predict(X_train))
test_acc = accuracy_score(y_test, logReg.predict(X_test))
summary.append({"Model": "Logistic Regression",  "Train Time (s)": train_time,"Train Accuracy": train_acc, "Test Accuracy": test_acc})
print("Logistic Regression\n",summary[-1])

Logistic Regression
 {'Model': 'Logistic Regression', 'Train Time (s)': 0.05209345801267773, 'Train Accuracy': 0.8967707996115247, 'Test Accuracy': 0.8980941976207818}


### Problem 9: Score the Model

What is the accuracy of your model?

In [19]:
# DECISION TREE CLASSIFIER
dtree = DecisionTreeClassifier()
start = time.perf_counter()
dtree.fit(X_train, y_train)
train_time = time.perf_counter() - start
train_acc = accuracy_score(y_train, dtree.predict(X_train))
test_acc = accuracy_score(y_test, dtree.predict(X_test))
summary.append({"Model": "Decision Tree Classifier",  "Train Time (s)": train_time,"Train Accuracy": train_acc, "Test Accuracy": test_acc})
print("Decision Tree Classifier\n",summary[-1])

Decision Tree Classifier
 {'Model': 'Decision Tree Classifier', 'Train Time (s)': 0.08557279093656689, 'Train Accuracy': 0.9705001618646811, 'Test Accuracy': 0.8363680504976936}


In [20]:
# K NEIGHBORS CLASSIFIER
knn = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier())])
start = time.perf_counter()
knn.fit(X_train, y_train)
train_time = time.perf_counter() - start
train_acc = accuracy_score(y_train, knn.predict(X_train))
test_acc = accuracy_score(y_test, knn.predict(X_test))
summary.append({"Model": "K Neighbors Classifier",  "Train Time (s)": train_time,"Train Accuracy": train_acc, "Test Accuracy": test_acc})
print("K Neighbors Classifier\n",summary[-1])

K Neighbors Classifier
 {'Model': 'K Neighbors Classifier', 'Train Time (s)': 0.01303804200142622, 'Train Accuracy': 0.9055519585626416, 'Test Accuracy': 0.8894149065307113}


In [21]:
# SUPPORT VECTOR CLASSIFIER
svc = SVC()
start = time.perf_counter()
svc.fit(X_train, y_train)
train_time = time.perf_counter() - start
train_acc = accuracy_score(y_train, svc.predict(X_train))
test_acc = accuracy_score(y_test, svc.predict(X_test))
summary.append({"Model": "Support Vector Machine",  "Train Time (s)": train_time,"Train Accuracy": train_acc, "Test Accuracy": test_acc})
print("Support Vector Machine\n",summary[-1])

Support Vector Machine
 {'Model': 'Support Vector Machine', 'Train Time (s)': 2.587900999933481, 'Train Accuracy': 0.896973130462933, 'Test Accuracy': 0.8983369749939306}


In [22]:
summary_df = pd.DataFrame(summary).sort_values(by="Test Accuracy", ascending=False)
print("Baseline Accuracy:- [", baseline_accuracy, "]\nWith Features :- ", features,"\n[Summary]\n", summary_df)

Baseline Accuracy:- [ 0.8873458288821987 ]
With Features :-  ['age', 'njob', 'nmarital', 'neducation', 'campaign', 'pdays', 'previous', 'ndefault', 'nhousing', 'nloan', 'ncontact', 'npoutcome'] 
[Summary]
                       Model  Train Time (s)  Train Accuracy  Test Accuracy
3    Support Vector Machine        2.587901        0.896973       0.898337
0       Logistic Regression        0.052093        0.896771       0.898094
2    K Neighbors Classifier        0.013038        0.905552       0.889415
1  Decision Tree Classifier        0.085573        0.970500       0.836368


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

- Baseline Accuracy:- [ 0.8873458288821987 ]
- with Features { ['age', 'njob', 'nmarital', 'neducation', 'campaign', 'pdays', 'previous', 'ndefault', 'nhousing', 'nloan', 'ncontact', 'npoutcome'] }

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|   SVM   |  2.587901  |. 0.896973    |.  0.898337   |
|   LogisticRegression  |  0.052093  |. 0.896771    |.  0.898094   |
|   KNeighborsClassifier  |  0.013038  |. 0.905552    |.  0.889415  |
|   DecisionTreeClassifier  |  0.085573  |. 0.970500    |.  0.836368   |

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [23]:
#Let's find out feature importances using RandomForestClassifier

rf = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("model", RandomForestClassifier(n_estimators=120, max_depth=20, n_jobs=-1, random_state=42))
])
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("RandomForestClassifier Accuracy: ", accuracy)

# Get importances
rf_importances = rf.named_steps["model"].feature_importances_*100
rf_importance_df = (
    pd.DataFrame({"feature": features, "importance": rf_importances}).sort_values("importance", ascending=False)
)
print(rf_importance_df)


RandomForestClassifier Accuracy:  0.8917212915756252
       feature  importance
0          age   32.823595
5        pdays   11.978918
1         njob   11.787892
4     campaign   11.166995
3   neducation    9.526892
2     nmarital    4.541367
8     nhousing    4.174811
6     previous    3.591788
11   npoutcome    3.416865
9        nloan    3.198549
10    ncontact    2.370047
7     ndefault    1.422281


In [24]:
#Chose top 5 features based on feature importance
improved_features = ["age", "pdays", "njob", "campaign", "neducation"]
improved_X = ndf[improved_features].copy()
improved_y = ndf["nsubscribed"].copy()
improved_X_train, improved_X_test, improved_y_train, improved_y_test = train_test_split(improved_X, improved_y, random_state=42, stratify=improved_y, test_size=0.4)

In [25]:
# Lets do Hyperparameter Tuning using GridSearchCV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grids = {
    "logreg": {
        "model": Pipeline([
            ("scaler", StandardScaler()),
            ("clf", LogisticRegression(max_iter=1000))
        ]),
        "params": {
            "clf__C": [0.01, 0.1, 1, 10, 100],
            # "clf__penalty": ["l2"],
            "clf__solver": ["lbfgs", "liblinear"]
        }
    },
    "dtree": {
        "model": DecisionTreeClassifier(random_state=42),
        "params": {
            "max_depth": [None, 3, 5, 7, 10],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4],
            "criterion": ["gini", "entropy"]
        }
    },
    "knn": {
        "model": Pipeline([
            ("scaler", StandardScaler()),
            ("clf", KNeighborsClassifier())
        ]),
        "params": {
            "clf__n_neighbors": [3, 5, 7, 9, 11],
            "clf__weights": ["uniform", "distance"],
            "clf__metric": ["minkowski", "manhattan"]
        }
    },
    "svc": {
        "model": Pipeline([
            ("scaler", StandardScaler()),
            ("clf", SVC())
        ]),
        "params": {
            "clf__C": [0.1, 1, 10],
            "clf__kernel": ["rbf"],
            "clf__gamma": ["scale"]
        }
    }
}

results = {}
summary_hyperparam_tuning = []
for name, cfg in grids.items():
    gs = GridSearchCV(
        estimator=cfg["model"],
        param_grid=cfg["params"],
        cv=cv,
        scoring="accuracy",
        n_jobs=-1,
        return_train_score=True
    )
    start = time.perf_counter()
    gs.fit(improved_X_train, improved_y_train)
    train_time = time.perf_counter() - start
    results[name] = gs
    print(f"{name}: best CV acc={gs.best_score_:.4f}, best params: {gs.best_params_}")
    train_acc = accuracy_score(improved_y_train, gs.predict(improved_X_train))
    test_acc = accuracy_score(improved_y_test, gs.predict(improved_X_test))
    summary_hyperparam_tuning.append({"Model": name,  "Train Time (s)": train_time,"Train Accuracy": train_acc, "Test Accuracy": test_acc, "Best Score": gs.best_score_})
    summary_hyperparam_tuning_df = pd.DataFrame(summary_hyperparam_tuning).sort_values(by="Test Accuracy", ascending=False)

# Best estimator overall
best_name = max(results, key=lambda k: results[k].best_score_)
best_model = results[best_name].best_estimator_
print("Best overall:", best_name, results[best_name].best_score_)

logreg: best CV acc=0.8971, best params: {'clf__C': 0.1, 'clf__solver': 'lbfgs'}
dtree: best CV acc=0.8971, best params: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}
knn: best CV acc=0.8947, best params: {'clf__metric': 'minkowski', 'clf__n_neighbors': 11, 'clf__weights': 'uniform'}
svc: best CV acc=0.8973, best params: {'clf__C': 10, 'clf__gamma': 'scale', 'clf__kernel': 'rbf'}
Best overall: svc 0.8973373348115092


## Grid Search Results

The table below summarizes cross-validated results from the hyperparameter search.

Model | Best CV Accuracy | Best Params
--- | ---: | ---
**Decision Tree** | 0.8971 | `{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}`
**Logistic Regression** | 0.8971 | `{'clf__C': 0.1, 'clf__solver': 'lbfgs'}`
**K-Nearest Neighbors** | 0.8945 | `{'clf__metric': 'minkowski', 'clf__n_neighbors': 11, 'clf__weights': 'uniform'}`
**Support Vector Machine** | 0.8973 | `{'clf__C': 10, 'clf__gamma': 'scale', 'clf__kernel': 'rbf'}`

**Best overall:** **Support Vector Machine** — CV accuracy: **0.8973373348115092**


##### Questions